---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
'''
Returns a DataFrame of towns and the states they are in from the 
university_towns.txt list. The format of the DataFrame should be:
DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
columns=["State", "RegionName"]  )
    
The following cleaning needs to be done:

1. For "State", removing characters from "[" to the end.
2. For "RegionName", when applicable, removing every character from " (" to the end.
3. Depending on how you read the data, you may need to remove newline character '\n'. 
'''

df = pd.read_table('https://storage.googleapis.com/um_ds_intro/university_towns.txt', header=None)
df[0] = df[0].str.replace('edit', "")
df[0] = df[0].str.replace('[', "")
df[0] = df[0].str.replace(']', "")
df[0] = df[0].str.replace('[0-9]', "")

# identify the state (in lines that do not contain '()')
df['State'] = df[0]
df['State'][df['State'].str.contains('[(]')] = np.NaN

# Spread the state values
for i in range(0,(len(df))):
    try:
        if np.isnan(df['State'][i]) == True:
            df['State'][i] = df['State'][i-1]
    except: 
        continue
# remove states from '0'
df = df[df[0] != df['State']]
df.set_index('State', inplace = True)
#df.columns = ['College_town']

# College
df['College'] = df[0].str.split('[(]').str[0]
df['College_town'] = df[0].str.split('[(]').str[1]
df['College_town'] = df['College_town'].str.replace('[)]',"")
del df[0]
#df

In [4]:
'''
Returns the year and quarter of the recession start time as a 
string value in a format such as 2005q3
'''

gdp_df = pd.read_excel('gdplev.xls', skiprows=5)

del gdp_df['Unnamed: 0']
del gdp_df['GDP in billions of current dollars'] 
del gdp_df['GDP in billions of chained 2009 dollars']
del gdp_df['Unnamed: 3']
del gdp_df['Unnamed: 7']
gdp_df.columns =["Quarter",
                 "GDP_bill_curnt",
                 "GDP_bill_2009"]
del gdp_df['GDP_bill_curnt']
gdp_df = gdp_df.dropna(how = 'any')

# Keep only from 2000:
gdp_df['year'] =gdp_df['Quarter'].str[0:4].astype('int')
gdp_df = gdp_df[gdp_df['year']>=2000]

# Define recession
"""
1. A recession is defined as starting with two consecutive quarters of GDP 
decline, and ending with two consecutive quarters of GDP growth.
2. A recession bottom is the quarter within a recession which had the lowest GDP.
"""
gdp_df = gdp_df.reset_index()

recession_times = list([])
for j in range(2,len(gdp_df)):
    if gdp_df['GDP_bill_2009'][j-1]<gdp_df['GDP_bill_2009'][j-2]: 
         if gdp_df['GDP_bill_2009'][j]<gdp_df['GDP_bill_2009'][j-1]:
                quarter = gdp_df['Quarter'][j]
                recession_times.append(quarter)
recession_times = pd.DataFrame(recession_times)
recession_times.columns = ['recession']
recession_times['recession_dum'] = 1

gdp_df = pd.merge(gdp_df,
                  recession_times,
                  how = 'left', 
                  left_on = 'Quarter',
                  right_on = 'recession')
del gdp_df['recession']
gdp_df['recession_dum'][np.isnan(gdp_df['recession_dum'])] =0 
# Define recession bottom
# Note: by defenition of recession, the bottom would be the last period (since
# we have only one recession)
gdp_df['recession_bottom1'] = gdp_df.groupby('recession_dum')['GDP_bill_2009'].transform('min')
gdp_df['recession_bottom'] = 0
gdp_df['recession_bottom'][(gdp_df['GDP_bill_2009'] == gdp_df['recession_bottom1']) &
                           (gdp_df['recession_dum'] == 1)] = 1

del gdp_df['recession_bottom1']


In [5]:
'''
Returns the year and quarter of the recession end time as a 
string value in a format such as 2005q3
'''
after_recession = list([])
for j in range(1, len(gdp_df)):
    if (gdp_df['recession_dum'][j-1] == 1) & (gdp_df['recession_dum'][j] == 0):
        after = gdp_df['Quarter'][j]
        after_recession.append(after)
after_recession = pd.DataFrame(after_recession)
after_recession[0][0]

'2009q3'

In [6]:
'''
Converts the housing data to quarters and returns it as mean 
values in a dataframe. This dataframe should be a dataframe with
columns for 2000q1 through 2016q3, and should have a multi-index
in the shape of ["State","RegionName"].
    
Note: Quarters are defined in the assignment description, they are
not arbitrary three month periods.
    
The resulting dataframe should have 67 columns, and 10,730 rows.
'''
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

relevent_months = []
for y in range(2000,2017):
    if y == 2016:
        for m in ['01', '02', '03', '04', '05', '06', '07', '08']:
            month = [str(y),"-",m]
            month = ''.join(month)
            relevent_months.append(month)
    else:
        for m in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
            month = [str(y),"-",m]
            month = ''.join(month)
            relevent_months.append(month)
columns = ['RegionID', 'RegionName', 'State', 'Metro', 
           'CountyName', 'SizeRank', relevent_months]
columns=flatten(columns)
columns_thin = ['RegionID', 'RegionName', 'State', 'Metro', 
           'CountyName', 'SizeRank']

housing_df = pd.read_csv('City_Zhvi_AllHomes.csv', usecols = columns)
housing_df_thin = pd.read_csv('City_Zhvi_AllHomes.csv', usecols = columns_thin)

# Transform housing_df into quarterly df

for y in range(2000, 2016): 
    y = str(y)
    matching_year = [s for s in housing_df.columns if y in s]
    df_temp = housing_df[matching_year]
    # Q1
    match1 = [s for s in df_temp.columns if '-01' in s]
    match2 = [s for s in df_temp.columns if '-02' in s]
    match3 = [s for s in df_temp.columns if '-03' in s]
    q1_list = flatten([match1, match2, match3])
    Q1_temp = df_temp[q1_list]
    housing_df_thin['_'.join(['Q1',y,'mean'])] = np.mean(Q1_temp, axis = 1)
    # Q2
    match1 = [s for s in df_temp.columns if '-04' in s]
    match2 = [s for s in df_temp.columns if '-05' in s]
    match3 = [s for s in df_temp.columns if '-06' in s]
    q2_list = flatten([match1, match2, match3])
    Q2_temp = df_temp[q2_list]
    housing_df_thin['_'.join(['Q2',y,'mean'])] = np.mean(Q2_temp, axis = 1)
    
    # Q3
    match1 = [s for s in df_temp.columns if '-07' in s]
    match2 = [s for s in df_temp.columns if '-08' in s]
    match3 = [s for s in df_temp.columns if '-09' in s] #
    q3_list = flatten([match1, match2, match3])
    Q3_temp = df_temp[q3_list]
    housing_df_thin['_'.join(['Q3',y,'mean'])] = np.mean(Q3_temp, axis = 1)

    # Q4
    match1 = [s for s in df_temp.columns if '-10' in s]
    match2 = [s for s in df_temp.columns if '-11' in s]
    match3 = [s for s in df_temp.columns if '-12' in s]
    q4_list = flatten([match1, match2, match3])
    Q4_temp = df_temp[q4_list]
    housing_df_thin['_'.join(['Q4',y,'mean'])] = np.mean(Q4_temp, axis = 1)
      
# 2016
y = 2016
y = str(y)
matching_year = [s for s in housing_df.columns if y in s]
df_temp = housing_df[matching_year]
# Q1
match1 = [s for s in df_temp.columns if '-01' in s]
match2 = [s for s in df_temp.columns if '-02' in s]
match3 = [s for s in df_temp.columns if '-03' in s]
q1_list = flatten([match1, match2, match3])
Q1_temp = df_temp[q1_list]
housing_df_thin['_'.join(['Q1',y,'mean'])] = np.mean(Q1_temp, axis = 1)
# Q2
match1 = [s for s in df_temp.columns if '-04' in s]
match2 = [s for s in df_temp.columns if '-05' in s]
match3 = [s for s in df_temp.columns if '-06' in s]
q2_list = flatten([match1, match2, match3])
Q2_temp = df_temp[q2_list]
housing_df_thin['_'.join(['Q2',y,'mean'])] = np.mean(Q2_temp, axis = 1)
    
# Q3
match1 = [s for s in df_temp.columns if '-07' in s]
match2 = [s for s in df_temp.columns if '-08' in s]
q3_list = flatten([match1, match2])
Q3_temp = df_temp[q3_list]
housing_df_thin['_'.join(['Q3',y,'mean'])] = np.mean(Q3_temp, axis = 1)

        
#del housing_df_thin['Q4_2016_mean']


In [7]:
housing_df_thin.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,Q1_2000_mean,Q2_2000_mean,Q3_2000_mean,Q4_2000_mean,...,Q2_2014_mean,Q3_2014_mean,Q4_2014_mean,Q1_2015_mean,Q2_2015_mean,Q3_2015_mean,Q4_2015_mean,Q1_2016_mean,Q2_2016_mean,Q3_2016_mean
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
2,17426,Chicago,IL,Chicago,Cook,3,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
5,18959,Las Vegas,NV,Las Vegas,Clark,6,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
6,54296,San Diego,CA,San Diego,San Diego,7,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
7,38128,Dallas,TX,Dallas-Fort Worth,Dallas,8,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
8,33839,San Jose,CA,San Jose,Santa Clara,9,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0
9,25290,Jacksonville,FL,Jacksonville,Duval,10,8.860000e+04,8.970000e+04,9.170000e+04,9.310000e+04,...,1.207667e+05,1.217333e+05,1.231667e+05,1.241667e+05,1.269000e+05,1.301333e+05,1.320000e+05,1.339667e+05,1.372000e+05,139900.0


In [ ]:
'''
First creates new data showing the decline or growth of housing prices
between the recession start and the recession bottom. Then runs a ttest
comparing the university town values to the non-university towns values, 
return whether the alternative hypothesis (that the two groups are the same)
is true or not as well as the p-value of the confidence. 

Return the tuple (different, p, better) where different=True if the t-test is
True at a p<0.01 (we reject the null hypothesis), or different=False if 
otherwise (we cannot reject the null hypothesis). The variable p should
be equal to the exact p value returned from scipy.stats.ttest_ind(). The
value for better should be either "university town" or "non-university town"
depending on which has a lower mean price ratio (which is equivilent to a
reduced market loss).
'''

'''
Note: There is a problem with the college_df indecies which is preventing me
        from merging. Therefore I can not constract a merged df and conduct
        the last question in this PS.
'''
# merging college file state names with the two-letters dictionary
states_df = pd.DataFrame([states]).T
states_df.reset_index(inplace = True)
states_df.columns = ['Short Name', 'Long Name']
college_df = pd.merge(df, states_df,
                     left_index = True,
                     right_on = 'Long Name',
                     how = 'left')
college_df.columns = ['College_town', 'College', 'Short Name', 'Long Name']
college_df.set_index(['Short Name', 'College_town'], inplace = True)
print('There are ', df.shape[0]-college_df['Short Name'].count(), 'that did not merge')
college_df.head()


In [22]:
housing_df_thin.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,Q1_2000_mean,Q2_2000_mean,Q3_2000_mean,Q4_2000_mean,...,Q2_2014_mean,Q3_2014_mean,Q4_2014_mean,Q1_2015_mean,Q2_2015_mean,Q3_2015_mean,Q4_2015_mean,Q1_2016_mean,Q2_2016_mean,Q3_2016_mean
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,207066.666667,214466.666667,220966.666667,226166.666667,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
2,17426,Chicago,IL,Chicago,Cook,3,138400.000000,143633.333333,147866.666667,152133.333333,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,53000.000000,53633.333333,54133.333333,54700.000000,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,111833.333333,114366.666667,116000.000000,117400.000000,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [94]:
housing_df_thin[(housing_df_thin['State'] == 'CA') & (housing_df_thin['RegionName'] == 'Orange')]

,RegionID,RegionName,State,Metro,CountyName,SizeRank,Q1_2000_mean,Q2_2000_mean,Q3_2000_mean,Q4_2000_mean,...,Q2_2014_mean,Q3_2014_mean,Q4_2014_mean,Q1_2015_mean,Q2_2015_mean,Q3_2015_mean,Q4_2015_mean,Q1_2016_mean,Q2_2016_mean,Q3_2016_mean
207,33252,Orange,CA,Los Angeles-Long Beach-Anaheim,Orange,208,255133.333333,261766.666667,270533.333333,278100.0,...,556333.333333,563200.0,568733.333333,576933.333333,588400.0,598900.0,604433.333333,612966.666667,621933.333333,630650.0


In [95]:
college_df[(college_df['Short Name'] == 'CA') & (college_df['College_town'] == 'Orange')]

,College_town,College,Short Name,Long Name
